In [1]:
import pandas as pd
import numpy as np
import io
import os
import re
from google.colab import drive
!pip install transformers
!pip install SentencePiece
import torch
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import defaultdict
from textwrap import wrap
from pylab import rcParams

from torch import nn, optim
from tqdm.notebook import tqdm
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from transformers import (AutoConfig, 
                          AutoModelForSequenceClassification, 
                          AutoTokenizer, AdamW, 
                          get_linear_schedule_with_warmup,
                          set_seed,
                          )
from keras.preprocessing.sequence import pad_sequences
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
set_seed(123)
epochs = 4
batches = 10
max_len = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'bert-base-cased'
# model_name = 'xlnet-base-cased'
# model_name = 'xlm-roberta-large'
n_labels = 3
class_names = ['negative', 'positive','neutral']
output_model = 'stocknews.bin'

In [3]:
path = '/content/drive/MyDrive/colab_data'
def de_emojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text
def text_proc(df, text_col='text'):
    df['orig_text'] = df[text_col]
    # Remove twitter handles
    df[text_col] = df[text_col].apply(lambda x: clean_text(x))
    # Remove URLs
    df[text_col] = df[text_col].apply(lambda x:x.replace('<br />', ' '))
    return df[df[text_col]!='']

In [4]:
data = pd.read_csv(os.path.join(path, "nasdaq.csv"))
data = text_proc(data,'Headline').dropna(subset=['Label'])#.sample(2000, random_state = 10).reset_index(drop=True)
data['len'] = data.Headline.apply(lambda x: len(x.split(' ')))
print(len(data))
print(data['len'].quantile(0.99))
data.head(3)

13181
52.0


,Label,Ticker,Headline,orig_text,len
0,0,A,Airline shares tumble as New York imposes qua...,@TotesTravel : Airline shares tumble as New Yo...,35
1,1,A,American United call off Hong Kong flights af...,@TotesTravel : American United call off Hong K...,35
2,0,A,U.S. airline stocks hit highest prices since ...,@TotesTravel : U.S. airline stocks hit highest...,37


In [5]:
df_train, df_val = train_test_split(data, test_size=0.33, random_state=42)

In [6]:
class Transformer_Dataset(Dataset):

    def __init__(self, text, target, tokenizer, max_len):
        self.text = text
        self.target = target
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item):
        text = str(self.text[item])
        target = self.target[item]

        encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        pad_to_max_length=False,
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
        )

        input_ids = pad_sequences(encoding['input_ids'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 

        attention_mask = pad_sequences(encoding['attention_mask'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       

        return {
        'text': text,
        'input_ids': input_ids,
        'attention_mask': attention_mask.flatten(),
        'target': torch.tensor(target, dtype=torch.long)
        }

def create_data_loader(df, text, target, tokenizer, max_len, batch_size):
  ds = Transformer_Dataset(
    text=df[text].to_numpy(),
    target=df[target].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2,
    drop_last = True
  )

# Get model configuration.
print('Loading configuraiton...')
model_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_name, 
                                          num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Get the actual model.
print('Loading model...')
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name, 
                                                           config=model_config)

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)



Loading configuraiton...
Loading tokenizer...
Loading model...


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Model loaded to `cuda`


In [7]:
train_data_loader = create_data_loader(df_train,'Headline', 'Label', tokenizer, max_len, batches)
val_data_loader = create_data_loader(df_val, 'Headline', 'Label', tokenizer, max_len, batches)

In [8]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)

total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

In [9]:
from sklearn import metrics
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples, batch_size, maxlen):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
  
    for d in data_loader:
        input_ids = d["input_ids"].reshape(batch_size,maxlen).to(device)
        attention_mask = d["attention_mask"].to(device)
        target = d['target'].to(device)
        
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = target)
        loss = outputs[0]
        logits = outputs[1]

        # preds = preds.cpu().detach().numpy()
        _, prediction = torch.max(outputs[1], dim=1)
        target = target.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(target, prediction)

        acc += accuracy
        losses.append(loss.item())
        
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        counter = counter + 1

    return acc / counter, np.mean(losses)

def eval_model(model, data_loader, device, n_examples,batch_size,maxlen):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
  
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(batch_size,maxlen).to(device)
            attention_mask = d["attention_mask"].to(device)
            target = d['target'].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = target)
            loss = outputs[0]
            logits = outputs[1]

            _, prediction = torch.max(outputs[1], dim=1)
            target = target.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(target, prediction)

            acc += accuracy
            losses.append(loss.item())
            counter += 1

    return acc / counter, np.mean(losses)

In [10]:
%%time
history = defaultdict(list)
best_accuracy = 0

for epoch in tqdm(range(epochs)):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,     
        optimizer, 
        device, 
        scheduler, 
        len(df_train),
        batches,
        max_len
    )

    print(f'Train loss {train_loss} Train accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader, 
        device, 
        len(df_val),
        batches,
        max_len
    )

    print(f'Val loss {val_loss} Val accuracy {val_acc}')
    print()

    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), os.path.join(path, output_model))
        best_accuracy = val_acc

Epoch 1/4
----------
Train loss 0.743994872574088 Train accuracy 0.6449603624009066
Val loss 0.7336052577385958 Val accuracy 0.6452873563218381

Epoch 2/4
----------
Train loss 0.6431200127639425 Train accuracy 0.695583238958097
Val loss 0.7198057700847758 Val accuracy 0.6779310344827582

Epoch 3/4
----------
Train loss 0.4273636349996853 Train accuracy 0.8319365798414443
Val loss 0.8817875334072387 Val accuracy 0.6889655172413786

Epoch 4/4
----------
Train loss 0.3061764814700016 Train accuracy 0.8958097395243428
Val loss 1.1951032187661221 Val accuracy 0.6836781609195401


CPU times: user 7min 44s, sys: 51.9 s, total: 8min 36s
Wall time: 8min 45s


Tried ```bert-base-cased```, val accuracy convert after epoch 1, accuracy in validation dataset is 0.6889655172413786.

Tried ```xlnet-base-cased```, val accuracy convert after epoch 1, accuracy in validation dataset is 0.6452873563218381

In [12]:
def predict_sentiment(text):
    text = text

    encoded_review = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=max_len,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
    )

    input_ids = pad_sequences(encoded_review['input_ids'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoded_review['attention_mask'], maxlen=max_len, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask) 

    input_ids = input_ids.reshape(1,max_len).to(device)
    attention_mask = attention_mask.to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    outputs = outputs[0][0].cpu().detach()

    probs = F.softmax(outputs, dim=-1).cpu().detach().numpy().tolist()
    _, prediction = torch.max(outputs, dim =-1)
    # return class_names[prediction]
    return prediction
    # print("Positive score:", probs[1])
    # print("Negative score:", probs[0])
    # print("Neutral score", probs[2])
    # print(f'text: {text}')
    # print(f'target  : {class_names[prediction]}')

In [52]:
df_val.head(3)

,Label,Ticker,Headline,orig_text,len
10734,0,AAN,S Africa 49 starving monkeys rescued from cage...,S Africa: 49 starving monkeys rescued from cag...,41
6086,0,ASET,China Russia flex muscles in increasing number...,China Russia flex muscles in increasing number...,13
7600,0,HWKN,Stephen Hawkins talks about his biggest myster...,Stephen Hawkins talks about his biggest myster...,8


In [54]:
df_val.Label.value_counts()

0    2807
1    1438
2     105
Name: Label, dtype: int64

In [27]:
predict_sentiment(df_val.Headline[10734])

Positive score: 0.04378170892596245
Negative score: 0.9514105916023254
Neutral score 0.004807799123227596
text: S Africa 49 starving monkeys rescued from cages Starving dehydrated monkeys were rescued from garden nursery near Johannesburg Saturday kept for the entertainment of those buying plants. Some heads were disproportionally large for their bodies amp bones protruded. Many had rickets.
target  : negative


In [13]:
df_val['pred']=df_val.Headline.apply(lambda x: int(predict_sentiment(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
df_val.head(3)

,Label,Ticker,Headline,orig_text,len,pred
10734,0,AAN,S Africa 49 starving monkeys rescued from cage...,S Africa: 49 starving monkeys rescued from cag...,41,0
6086,0,ASET,China Russia flex muscles in increasing number...,China Russia flex muscles in increasing number...,13,0
7600,0,HWKN,Stephen Hawkins talks about his biggest myster...,Stephen Hawkins talks about his biggest myster...,8,0


In [15]:
confusion_matrix(df_val.Label, df_val.pred)

array([[2217,  566,   24],
       [ 697,  732,    9],
       [  46,   34,   25]])

In [16]:
print(classification_report(df_val.Label, df_val.pred))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77      2807
           1       0.55      0.51      0.53      1438
           2       0.43      0.24      0.31       105

    accuracy                           0.68      4350
   macro avg       0.58      0.51      0.53      4350
weighted avg       0.68      0.68      0.68      4350

